# LAB 06: Advanced Transforms -- PySpark & SQL

**Duration:** ~40 min | **Day:** 2 | **Difficulty:** Intermediate-Advanced
**After module:** M06: Advanced Transforms

> *"Build analytical reports using window functions, CTEs, explode, and CTAS."*

## Setup

In [ ]:
%run ../../setup/00_setup

In [ ]:
from pyspark.sql.functions import col, sum, count, desc, row_number, rank, dense_rank, lag, lead, explode, from_json
from pyspark.sql.window import Window

# Load base data
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{GOLD_SCHEMA}")

df_orders = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.orders")
df_customers = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers")
df_products = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.products")

# Register as temp views for SQL tasks
df_orders.createOrReplaceTempView("orders")
df_customers.createOrReplaceTempView("customers")
df_products.createOrReplaceTempView("products")

print(f"Data loaded: {df_orders.count()} orders, {df_customers.count()} customers, {df_products.count()} products")

---
## Task 1: Window Function -- Rank Products by Revenue (PySpark)

For each product, calculate total revenue. Then rank products using `row_number()`.

Hint: Use `Window.orderBy(desc("total_revenue"))`

In [ ]:
# TODO: Calculate total revenue per product and rank them
df_product_revenue = (
    df_orders
    .groupBy("product_id")
    .agg(sum("total_price").alias("total_revenue"))
)

window_spec = Window.orderBy(________("total_revenue"))

df_ranked = (
    df_product_revenue
    .withColumn("rank", ________(________))
)

display(df_ranked.limit(10))

In [ ]:
# -- Validation --
assert "rank" in df_ranked.columns, "Missing 'rank' column"
first = df_ranked.orderBy("rank").first()
assert first["rank"] == 1, "First row should have rank 1"
print(f"Task 1 OK: Top product has revenue {first['total_revenue']:.2f}")

---
## Task 2: Running Total (SQL)

Write a SQL query to compute a cumulative running total per customer ordered by order_date.

In [ ]:
# TODO: Complete the SQL window function
df_running = spark.sql("""
    SELECT 
        customer_id,
        order_date,
        total_price,
        SUM(total_price) OVER (
            PARTITION BY ________
            ORDER BY ________
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS running_total
    FROM orders
    ORDER BY customer_id, order_date
""")

display(df_running.limit(20))

In [ ]:
# -- Validation --
assert "running_total" in df_running.columns, "Missing 'running_total' column"
print(f"Task 2 OK: Running totals computed for {df_running.select('customer_id').distinct().count()} customers")

---
## Task 3: Multi-step CTE

Write a SQL query with two CTEs to find the top 5 days by total revenue:
1. `daily_sales` -- total revenue per day
2. `ranked_days` -- rank days by revenue

In [ ]:
# TODO: Complete the CTE query
df_top_days = spark.sql("""
    WITH daily_sales AS (
        SELECT 
            order_date,
            ________(total_price) AS daily_revenue,
            ________(*)          AS order_count
        FROM orders
        GROUP BY order_date
    ),
    ranked_days AS (
        SELECT *,
            ROW_NUMBER() OVER (ORDER BY daily_revenue ________) AS day_rank
        FROM daily_sales
    )
    SELECT * FROM ranked_days
    WHERE day_rank <= 5
    ORDER BY day_rank
""")

display(df_top_days)

In [ ]:
# -- Validation --
assert df_top_days.count() <= 5, "Should return at most 5 rows"
assert df_top_days.first()["day_rank"] == 1, "First row should be rank 1"
print(f"Task 3 OK: Top {df_top_days.count()} days by revenue")

---
## Task 4: Correlated Subquery

Find customers whose total spending is above the overall average spending per customer.

In [ ]:
# TODO: Write SQL with subquery
df_high_spenders = spark.sql("""
    SELECT customer_id, SUM(total_price) AS total_spent
    FROM orders
    GROUP BY customer_id
    HAVING SUM(total_price) > (
        SELECT ________(total_spent) FROM (
            SELECT customer_id, SUM(total_price) AS total_spent
            FROM orders
            GROUP BY customer_id
        )
    )
    ORDER BY total_spent DESC
""")

display(df_high_spenders)

In [ ]:
# -- Validation --
assert df_high_spenders.count() > 0, "Should find at least some high spenders"
print(f"Task 4 OK: {df_high_spenders.count()} customers above average spending")

---
## Task 5: Explode Array Column

Create a sample DataFrame with an array column and use `explode()` to flatten it.

In [ ]:
from pyspark.sql.functions import array, lit, explode

# Sample data with array column
df_with_array = spark.createDataFrame([
    (1, ["Electronics", "Books", "Clothing"]),
    (2, ["Food", "Electronics"]),
    (3, ["Books"])
], ["customer_id", "categories"])

# TODO: Explode the categories array into individual rows
df_exploded = df_with_array.select(
    "customer_id",
    ________(col("categories")).alias("category")
)

display(df_exploded)

In [ ]:
# -- Validation --
assert df_exploded.count() == 6, f"Expected 6 rows after explode, got {df_exploded.count()}"
assert "category" in df_exploded.columns, "Missing 'category' column"
print(f"Task 5 OK: Exploded {df_with_array.count()} rows into {df_exploded.count()} rows")

---
## Task 6: CTAS -- Create Gold Tables

Use `CREATE TABLE AS SELECT` to persist the top products analysis as a Gold table.

In [ ]:
# TODO: Create gold table using CTAS
gold_table = f"{CATALOG}.{GOLD_SCHEMA}.top_products"
spark.sql(f"DROP TABLE IF EXISTS {gold_table}")

spark.sql(f"""
    ________ {gold_table} AS
    SELECT 
        product_id,
        SUM(total_price) AS total_revenue,
        COUNT(*) AS order_count
    FROM {CATALOG}.{BRONZE_SCHEMA}.orders
    GROUP BY product_id
    ORDER BY total_revenue DESC
    LIMIT 10
""")

display(spark.table(gold_table))

In [ ]:
# -- Validation --
gold_count = spark.table(gold_table).count()
assert gold_count > 0 and gold_count <= 10, f"Expected 1-10 rows, got {gold_count}"
detail = spark.sql(f"DESCRIBE DETAIL {gold_table}").first()
assert detail["format"] == "delta", "CTAS should create a Delta table"
print(f"Task 6 OK: Gold table '{gold_table}' created with {gold_count} rows")

---
### Task 7: JSON Processing -- `from_json` & `schema_of_json`

In practice, data often arrives as a STRING column containing JSON. Use `from_json()` with an explicit schema to parse JSON into a struct and extract fields.

**TODO:** Define the schema, parse JSON, and extract fields from the struct.

In [ ]:
from pyspark.sql.functions import from_json, col, schema_of_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Sample data with JSON string column
json_data = [
    (1, '{"product": "Laptop", "qty": 2, "price": 999.99}'),
    (2, '{"product": "Mouse", "qty": 5, "price": 29.99}'),
    (3, '{"product": "Monitor", "qty": 1, "price": 449.00}'),
]
df_json = spark.createDataFrame(json_data, ["order_id", "order_details"])

# TODO: Define the schema for the JSON string
json_schema = StructType([
    StructField("product", ________(), True),       # hint: StringType
    StructField("qty", ________(), True),            # hint: IntegerType
    StructField("price", ________(), True),          # hint: DoubleType
])

# TODO: Parse JSON string into a struct column using from_json
df_parsed = df_json.withColumn(
    "parsed",
    ________(col("order_details"), json_schema)      # hint: from_json
)

# TODO: Extract individual fields from the struct
df_extracted = df_parsed.select(
    "order_id",
    col("parsed.________").alias("product_name"),    # hint: product
    col("parsed.________").alias("quantity"),         # hint: qty
    col("parsed.________").alias("unit_price"),      # hint: price
    (col("parsed.qty") * col("parsed.price")).alias("line_total")
)

display(df_extracted)

In [ ]:
# -- Validation --
assert "product_name" in df_extracted.columns, "Missing 'product_name' — check col('parsed.product')"
assert "line_total" in df_extracted.columns, "Missing 'line_total' column"
assert df_extracted.count() == 3, f"Expected 3 rows, got {df_extracted.count()}"
first = df_extracted.filter("order_id = 1").first()
assert first["product_name"] == "Laptop", f"Expected 'Laptop', got '{first['product_name']}'"
assert abs(first["line_total"] - 1999.98) < 0.01, "line_total for Laptop should be ~1999.98"
print(f"Task 7 OK: JSON parsed and extracted — {df_extracted.count()} rows with {len(df_extracted.columns)} columns")

---
### Task 8: CASE WHEN -- Customer Segmentation (SQL)

Use `CASE WHEN` to segment customers based on their total spending:
- **VIP**: > 500
- **Regular**: 100–500
- **Low**: < 100

**TODO:** Fill in the CASE WHEN conditions.

In [ ]:
# TODO: Complete the CASE WHEN to categorize customers
df_segments = spark.sql("""
    SELECT 
        customer_id,
        SUM(total_price) AS total_spent,
        CASE
            WHEN SUM(total_price) > ________ THEN 'VIP'
            WHEN SUM(total_price) >= ________ THEN 'Regular'
            ELSE '________'
        END AS segment
    FROM orders
    GROUP BY customer_id
    ORDER BY total_spent DESC
""")

display(df_segments)

In [ ]:
# -- Validation --
assert "segment" in df_segments.columns, "Missing 'segment' column"
segments = [r["segment"] for r in df_segments.select("segment").distinct().collect()]
assert set(segments).issubset({"VIP", "Regular", "Low"}), f"Unexpected segments: {segments}"
seg_counts = {r["segment"]: r["count"] for r in df_segments.groupBy("segment").count().collect()}
print(f"Task 8 OK: Customer segments — {seg_counts}")

---
### Task 9: Higher-Order Functions -- `transform` & `filter`

Higher-Order Functions let you operate on array elements without needing `explode()`.

- `transform(arr, x -> expr)` — transforms each element
- `filter(arr, x -> condition)` — filters elements matching the condition

**TODO:** Use `transform` to convert prices and `filter` to find expensive products.

In [ ]:
from pyspark.sql.functions import array, lit, expr

# Sample data with price arrays (e.g. line items per order)
df_prices = spark.createDataFrame([
    (1, [29.99, 149.99, 9.99, 599.00]),
    (2, [49.99, 19.99]),
    (3, [999.99, 249.99, 79.99, 15.00, 450.00]),
], ["order_id", "item_prices"])

# TODO: Use transform to add 23% VAT to each price
# hint: transform(item_prices, x -> x * 1.23)
df_with_vat = df_prices.selectExpr(
    "order_id",
    "item_prices",
    "________( item_prices, x -> ROUND(x * 1.23, 2) ) AS prices_with_vat"
    # hint: transform
)

# TODO: Use filter to keep only items > 100
# hint: filter(prices_with_vat, x -> x > 100)
df_expensive = df_with_vat.selectExpr(
    "order_id",
    "prices_with_vat",
    "________( prices_with_vat, x -> x > 100 ) AS expensive_items"
    # hint: filter
)

display(df_expensive)

In [ ]:
# -- Validation --
assert "prices_with_vat" in df_expensive.columns, "Missing 'prices_with_vat' column"
assert "expensive_items" in df_expensive.columns, "Missing 'expensive_items' column"
# Verify VAT calculation: 29.99 * 1.23 = 36.89 (not > 100, so filtered out)
row1 = df_expensive.filter("order_id = 1").first()
vat_prices = row1["prices_with_vat"]
expensive = row1["expensive_items"]
assert len(vat_prices) == 4, f"Order 1 should have 4 VAT prices, got {len(vat_prices)}"
assert all(p > 100 for p in expensive), "All expensive items should be > 100"
print(f"Task 9 OK: transform applied VAT to {len(vat_prices)} items, filter kept {len(expensive)} expensive items")

---
## Lab Complete!

You have:
- Applied window functions (row_number, running SUM) in PySpark and SQL
- Written multi-step CTEs for complex analytics
- Used subqueries to filter by aggregate conditions
- Flattened arrays with explode()
- Created Gold tables using CTAS
- Parsed JSON strings with `from_json` and extracted struct fields
- Used `CASE WHEN` for customer segmentation
- Applied higher-order functions (`transform`, `filter`) on arrays

> **Exam Tip:** Know the difference: `ROW_NUMBER()` always gives unique sequential numbers. `RANK()` gives the same number for ties (with gaps). `DENSE_RANK()` gives same number for ties (no gaps).

> **Next:** LAB 07 - Build a Medallion Pipeline in Lakeflow

## Cleanup (Optional)

In [ ]:
# Optional cleanup
# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{GOLD_SCHEMA}.top_products")
print("LAB 06 complete.")